In [1]:
from FLAI import data
from FLAI import causal_graph
import pandas as pd
import numpy as np
import math

In [2]:
import time



In [4]:
for rows in [5000,10000,20000,50000,100000]:
    for n in [5,10,20,50,100]:

        df = pd.DataFrame()
        df['sex'] = np.random.randint(0,2,rows)
        features = []
        for n in range(n+1):
            df['feature_'+str(n)] = np.random.randint(0,10,rows)
            features.append('feature_'+str(n))
        df['label'] = np.random.randint(0,2,rows)

        flai_dataset = data.Data(df[features + ['sex', 'label']], transform=True)
        start_time = time.time()
        df_f,df_aux_ideal = flai_dataset.fairness_eqa_eqi(features = features, 
                                    target_column = 'label', 
                                    column_filter = ['sex'],
                                    plot = False)
        stop_time = time.time()
        duration = stop_time - start_time
        print('rows: ',rows,' - features: ',n,' - duration: ',duration)

rows:  5000  - features:  5  - duration:  0.046364784240722656
rows:  5000  - features:  10  - duration:  0.06594371795654297
rows:  5000  - features:  20  - duration:  0.16894817352294922
rows:  5000  - features:  50  - duration:  0.4366011619567871
rows:  5000  - features:  100  - duration:  0.9288651943206787
rows:  10000  - features:  5  - duration:  0.06342387199401855
rows:  10000  - features:  10  - duration:  0.08979535102844238
rows:  10000  - features:  20  - duration:  0.25887036323547363
rows:  10000  - features:  50  - duration:  0.6725640296936035
rows:  10000  - features:  100  - duration:  1.6034762859344482
rows:  20000  - features:  5  - duration:  0.09632611274719238
rows:  20000  - features:  10  - duration:  0.13683319091796875
rows:  20000  - features:  20  - duration:  0.4773087501525879
rows:  20000  - features:  50  - duration:  1.1816024780273438
rows:  20000  - features:  100  - duration:  2.7407267093658447
rows:  50000  - features:  5  - duration:  0.198116